In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('melbourne-housing-snapshot.zip')
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]
y = data.Price
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [6]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [5]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))


Mean Absolute Error: 240592.61237113402


#### Away from the sklearn framework

In [50]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, feature_names = cols_to_use, label = y_train)
dvalid = xgb.DMatrix(X_valid, feature_names = cols_to_use, label = y_valid)

In [15]:
params = my_model.get_xgb_params()
bst = xgb.train(params, dtrain)

In [16]:
preds = bst.predict(dvalid)

In [27]:
import numpy as np
from sklearn.metrics import r2_score
pred_labels = np.rint(preds)
r2 = r2_score(y_valid, pred_labels)

In [29]:
r2

0.6031711790809222

### Parameter Tuning with Optuna

In [56]:
def objective(trial, dtrain, dvalid, valid_y):
    param = {
    "verbosity": 0,
    "objective": "reg:squarederror",
    # use exact for small dataset.
    "tree_method": "hist",
    # defines booster, gblinear for linear functions.
    "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear"]),
    # L2 regularization weight.
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    # L1 regularization weight.
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    # sampling ratio for training data.
    "subsample": trial.suggest_float("subsample", 0.2, 1.0),
    }
    
    if param["booster"] == 'gbtree':
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    r2 = sklearn.metrics.r2_score(valid_y, pred_labels)
    return r2

In [57]:
from functools import partial

objective_with_data = partial(objective, dtrain = dtrain, dvalid = dvalid, valid_y = y_valid)

In [58]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_with_data, timeout=600)

[I 2023-03-31 11:57:15,743] A new study created in memory with name: no-name-9d46363c-bf56-44dc-8740-879c92ea0053
[I 2023-03-31 11:57:15,755] Trial 0 finished with value: 0.23315077569195708 and parameters: {'booster': 'gblinear', 'lambda': 0.006153238682948028, 'alpha': 4.6766064978994697e-07, 'subsample': 0.6736201860759763}. Best is trial 0 with value: 0.23315077569195708.
[I 2023-03-31 11:57:15,790] Trial 1 finished with value: -2.5498027417055114 and parameters: {'booster': 'gbtree', 'lambda': 1.564650236873472e-05, 'alpha': 0.06603366903398317, 'subsample': 0.6875468152195074, 'max_depth': 9, 'min_child_weight': 9, 'eta': 0.00167203171961695, 'gamma': 1.0625328396483264e-07, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.23315077569195708.
[I 2023-03-31 11:57:15,797] Trial 2 finished with value: -2.654849045643223 and parameters: {'booster': 'gbtree', 'lambda': 0.10590629208860854, 'alpha': 2.235906144386802e-06, 'subsample': 0.4770735179300749, 'max_depth': 3, 'min_c

[I 2023-03-31 11:57:16,232] Trial 23 finished with value: 0.3853730932332685 and parameters: {'booster': 'gbtree', 'lambda': 5.003250341722114e-08, 'alpha': 1.1316636935488993e-08, 'subsample': 0.7952218220448033, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.14031285637595312, 'gamma': 0.06967930255926165, 'grow_policy': 'depthwise'}. Best is trial 20 with value: 0.5975617868696614.
[I 2023-03-31 11:57:16,259] Trial 24 finished with value: 0.5519150067803613 and parameters: {'booster': 'gbtree', 'lambda': 3.730693642104057e-07, 'alpha': 5.712230869987002e-07, 'subsample': 0.8780192995560614, 'max_depth': 7, 'min_child_weight': 5, 'eta': 0.1873689965373386, 'gamma': 0.00038745519957854714, 'grow_policy': 'depthwise'}. Best is trial 20 with value: 0.5975617868696614.
[I 2023-03-31 11:57:16,280] Trial 25 finished with value: -2.3107493475979215 and parameters: {'booster': 'gbtree', 'lambda': 5.090383396710349e-08, 'alpha': 6.348606043430886e-08, 'subsample': 0.9213635146404098, 'max_de

[I 2023-03-31 11:57:16,789] Trial 45 finished with value: 0.5889316571466388 and parameters: {'booster': 'gbtree', 'lambda': 7.166596674424327e-08, 'alpha': 1.8136902109414808e-05, 'subsample': 0.8983075402285741, 'max_depth': 7, 'min_child_weight': 4, 'eta': 0.22125961664748905, 'gamma': 0.006829910765718968, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.6235755845180135.
[I 2023-03-31 11:57:16,802] Trial 46 finished with value: 0.23367204543565756 and parameters: {'booster': 'gblinear', 'lambda': 7.969375103825805e-08, 'alpha': 1.382295333256214e-05, 'subsample': 0.8998219696799152}. Best is trial 43 with value: 0.6235755845180135.
[I 2023-03-31 11:57:16,832] Trial 47 finished with value: 0.5883651476181615 and parameters: {'booster': 'gbtree', 'lambda': 4.7662343229699294e-07, 'alpha': 3.295175417465856e-05, 'subsample': 0.9346961110797498, 'max_depth': 7, 'min_child_weight': 4, 'eta': 0.21130724539445703, 'gamma': 0.011421834287625932, 'grow_policy': 'depthwise'}. Bes

[I 2023-03-31 11:57:17,502] Trial 66 finished with value: -0.6371301215590262 and parameters: {'booster': 'gbtree', 'lambda': 3.030473157815529e-07, 'alpha': 9.335246543248835e-05, 'subsample': 0.8767409583593139, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.04758538760947741, 'gamma': 0.05004586984002281, 'grow_policy': 'depthwise'}. Best is trial 50 with value: 0.6423117436690344.
[I 2023-03-31 11:57:17,541] Trial 67 finished with value: 0.6121398662122547 and parameters: {'booster': 'gbtree', 'lambda': 5.08707607474461e-07, 'alpha': 5.746288446079844e-05, 'subsample': 0.8595673002518023, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.6200287712326501, 'gamma': 0.10140503296630543, 'grow_policy': 'depthwise'}. Best is trial 50 with value: 0.6423117436690344.
[I 2023-03-31 11:57:17,579] Trial 68 finished with value: 0.43675583763452297 and parameters: {'booster': 'gbtree', 'lambda': 6.448492130916235e-07, 'alpha': 2.3545415785226425e-06, 'subsample': 0.9339321196578376, 'max_depth

[I 2023-03-31 11:57:18,290] Trial 87 finished with value: 0.30505605075163056 and parameters: {'booster': 'gbtree', 'lambda': 1.4273911570499713e-07, 'alpha': 8.984410441362932e-07, 'subsample': 0.9501141837123754, 'max_depth': 9, 'min_child_weight': 4, 'eta': 0.11267767225734474, 'gamma': 0.0397254332466047, 'grow_policy': 'depthwise'}. Best is trial 50 with value: 0.6423117436690344.
[I 2023-03-31 11:57:18,359] Trial 88 finished with value: 0.6316453460547091 and parameters: {'booster': 'gbtree', 'lambda': 8.837397088642829e-07, 'alpha': 4.656665899044357e-06, 'subsample': 0.8770251215546973, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.38687061856657773, 'gamma': 0.19510519314018546, 'grow_policy': 'lossguide'}. Best is trial 50 with value: 0.6423117436690344.
[I 2023-03-31 11:57:18,375] Trial 89 finished with value: 0.23326363678740736 and parameters: {'booster': 'gblinear', 'lambda': 1.9405256707641733e-06, 'alpha': 5.221454070307945e-06, 'subsample': 0.8149624737566239}. Best 

...

[I 2023-03-31 12:06:51,284] Trial 4287 finished with value: 0.3826634341322269 and parameters: {'booster': 'gbtree', 'lambda': 2.8332912757773326e-06, 'alpha': 1.2423844653266004e-05, 'subsample': 0.9218668871291864, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.12369108624467427, 'gamma': 0.01639478522825535, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:06:51,608] Trial 4288 finished with value: 0.5723149531769061 and parameters: {'booster': 'gbtree', 'lambda': 0.028072819867030852, 'alpha': 0.0015887109442025327, 'subsample': 0.3465466639956738, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.3335575537508502, 'gamma': 0.006502349403339339, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:06:51,817] Trial 4289 finished with value: 0.5733096781012715 and parameters: {'booster': 'gbtree', 'lambda': 0.0771061802930416, 'alpha': 0.6748031492061616, 'subsample': 0.8368115759769502, 'max_d

[I 2023-03-31 12:06:57,010] Trial 4308 finished with value: -0.14408132038002686 and parameters: {'booster': 'gbtree', 'lambda': 4.220134372780074e-08, 'alpha': 8.902662260042893e-08, 'subsample': 0.7403314541929923, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.0715403959334048, 'gamma': 8.765526337619467e-07, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:06:57,323] Trial 4309 finished with value: -2.6533670783798975 and parameters: {'booster': 'gbtree', 'lambda': 1.8239755597142566e-05, 'alpha': 0.07822562842806671, 'subsample': 0.601988502173149, 'max_depth': 9, 'min_child_weight': 10, 'eta': 2.3190886367583825e-05, 'gamma': 0.12400499148733962, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:06:57,639] Trial 4310 finished with value: 0.6375504663653611 and parameters: {'booster': 'gbtree', 'lambda': 0.002439163538358191, 'alpha': 0.49816149938566257, 'subsample': 0.9239248924647367

[I 2023-03-31 12:07:02,772] Trial 4329 finished with value: 0.6218418131540266 and parameters: {'booster': 'gbtree', 'lambda': 0.09780234213299824, 'alpha': 0.24595880223546965, 'subsample': 0.8044773620226113, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.3032927177905517, 'gamma': 0.012613604834160724, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:02,982] Trial 4330 finished with value: -2.4330709637451697 and parameters: {'booster': 'gbtree', 'lambda': 0.00045285605788101846, 'alpha': 0.00034257215272617543, 'subsample': 0.592113072176781, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.003573038897360015, 'gamma': 5.703478371112136e-07, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:03,290] Trial 4331 finished with value: 0.5934319864781454 and parameters: {'booster': 'gbtree', 'lambda': 0.0042141187277219445, 'alpha': 0.0005523790349300611, 'subsample': 0.9239549049393438, 

[I 2023-03-31 12:07:08,445] Trial 4350 finished with value: 0.6029242714103964 and parameters: {'booster': 'gbtree', 'lambda': 0.00010938402254478938, 'alpha': 3.701123237771598e-07, 'subsample': 0.8401650619474109, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.20769342455721598, 'gamma': 3.0391339780165542e-05, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:08,762] Trial 4351 finished with value: -2.654848418705295 and parameters: {'booster': 'gbtree', 'lambda': 0.0030816077882313696, 'alpha': 0.7367121211848665, 'subsample': 0.736086610903401, 'max_depth': 9, 'min_child_weight': 3, 'eta': 6.109554760789884e-08, 'gamma': 0.018342654591761213, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:08,969] Trial 4352 finished with value: 0.27777379644674705 and parameters: {'booster': 'gbtree', 'lambda': 1.275908448196007e-07, 'alpha': 8.897577634887253e-07, 'subsample': 0.589401650684948

[I 2023-03-31 12:07:14,077] Trial 4371 finished with value: -2.6548206494820583 and parameters: {'booster': 'gbtree', 'lambda': 0.0011167160733467536, 'alpha': 6.812447041650782e-05, 'subsample': 0.9996160637685172, 'max_depth': 9, 'min_child_weight': 3, 'eta': 4.878061949352808e-07, 'gamma': 0.4409527970528703, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:14,395] Trial 4372 finished with value: -2.6523234626312076 and parameters: {'booster': 'gbtree', 'lambda': 3.6679292396166767e-08, 'alpha': 0.11717219164702786, 'subsample': 0.9426976721941673, 'max_depth': 9, 'min_child_weight': 3, 'eta': 3.9411919930338075e-05, 'gamma': 2.346809044093833e-07, 'grow_policy': 'depthwise'}. Best is trial 1750 with value: 0.6562738256223292.
[I 2023-03-31 12:07:14,483] Trial 4373 finished with value: 0.23368255564941887 and parameters: {'booster': 'gblinear', 'lambda': 7.683375426253125e-08, 'alpha': 0.07281510097727878, 'subsample': 0.33213631511

In [59]:
study.best_trial.params

{'booster': 'gbtree',
 'lambda': 0.001656570003966527,
 'alpha': 0.7277157822832302,
 'subsample': 0.9544772053671777,
 'max_depth': 9,
 'min_child_weight': 3,
 'eta': 0.40866580549800857,
 'gamma': 1.1430459407894848e-05,
 'grow_policy': 'depthwise'}

In [60]:
xgb.train(study.best_params, dtrain)

In [61]:
preds = bst.predict(dvalid)

In [62]:
pred_labels = np.rint(preds)
r2 = r2_score(y_valid, pred_labels)

In [63]:
r2

0.6031711790809222

### Using CUDA